In [ ]:
import os
import re
import time

from langchain_core.documents import Document 
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_openai import OpenAIEmbeddings
from typing import List
from langchain_community.embeddings import HuggingFaceEmbeddings



from dotenv import load_dotenv

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("XAI_API_KEY")

e:\Anaconda3\envs\judge-assistant\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def split_egyptian_civil_law(text: str):
    """
    Splits Egyptian Civil Law text into Documents with types:
    - book, part, chapter, article
    - Avoids repeating books if they appear multiple times
    - Attaches hierarchy info in metadata: book, part, chapter
    - Supports preliminary باب (الباب التمهيدي)
    """
    # Heading patterns
    book_pat    = r"^الكتاب[^\n]*"
    part_pat    = r"^(الباب\s+التمهيدي|الباب[^\n]*)"
    chapter_pat = r"^الفصل[^\n]*"
    article_pat = r"^المادة\s*([0-9]+)"

    combined = rf"(?m)(?=({book_pat}|{part_pat}|{chapter_pat}|{article_pat}))"
    raw_sections = re.split(combined, text)
    sections = [s.strip() for s in raw_sections if s and s.strip()]

    docs = []
    last_heading_normalized = None
    seen_books = set()  # track unique books

    # Current hierarchy
    current_book = None
    current_part = None
    current_chapter = None

    re_book = re.compile(book_pat, flags=re.MULTILINE)
    re_part = re.compile(part_pat, flags=re.MULTILINE)
    re_chap = re.compile(chapter_pat, flags=re.MULTILINE)
    re_article = re.compile(article_pat, flags=re.MULTILINE)

    for sec in sections:
        m_article = re_article.match(sec)
        m_book = re_book.match(sec)
        m_part = re_part.match(sec)
        m_chap = re_chap.match(sec)

        # Handle articles
        if m_article:
            heading_raw = m_article.group(0).strip()
            heading_norm = heading_raw
            if heading_norm == last_heading_normalized:
                if docs:
                    docs[-1] = Document(
                        page_content=docs[-1].page_content + "\n" + sec[m_article.end():].strip(),
                        metadata=docs[-1].metadata
                    )
                continue
            last_heading_normalized = heading_norm

            body = sec[m_article.end():].strip()
            index = int(m_article.group(1))

            meta = {
                "type": "article",
                "title": heading_raw,
                "index": index,
                "book": current_book,
                "part": current_part,
                "chapter": current_chapter
            }
            docs.append(Document(page_content=f"{heading_raw}\n{body}", metadata=meta))
            continue

        # Handle books, parts, chapters
        if m_book or m_part or m_chap:
            m = m_book or m_part or m_chap
            heading_raw = m.group(0).strip()
            heading_norm = heading_raw
            if heading_norm == last_heading_normalized:
                if docs:
                    docs[-1] = Document(
                        page_content=docs[-1].page_content + "\n" + sec[m.end():].strip(),
                        metadata=docs[-1].metadata
                    )
                continue
            last_heading_normalized = heading_norm

            body = sec[m.end():].strip()

            if m is m_book:
                t = "book"
                # skip repeated books but keep hierarchy
                if heading_norm in seen_books:
                    current_book = heading_norm
                    current_part = None
                    current_chapter = None
                    continue
                seen_books.add(heading_norm)
                current_book = heading_raw
                current_part = None
                current_chapter = None
            elif m is m_part:
                t = "part"
                current_part = heading_raw
                current_chapter = None
            else:
                t = "chapter"
                current_chapter = heading_raw

            meta = {
                "type": t,
                "title": heading_raw,
                "book": current_book,
                "part": current_part,
                "chapter": current_chapter
            }
            docs.append(Document(page_content=f"{heading_raw}\n{body}", metadata=meta))
            continue

        # Any other text → preface
        if docs:
            docs[-1] = Document(
                page_content=docs[-1].page_content + "\n" + sec.strip(),
                metadata=docs[-1].metadata
            )
        else:
            docs.append(Document(page_content=sec.strip(), metadata={"type":"preface", "title":"المقدمة"}))

    return docs

In [3]:
current_dir = os.getcwd()
file_path = os.path.join(current_dir, "docs", "civil_law_clean.txt")
persistent_directory = os.path.join(current_dir, "db", "chroma_db")

# Ensure the text file exists
if not os.path.exists(file_path):
    raise FileNotFoundError(f"The file {file_path} does not exist. Please check the path.")

# Initialize embeddings (always)
embeddings = HuggingFaceEmbeddings(
    model_name="sayed0am/arabic-english-bge-m3"
)

# Initialize vector store, whether it exists or not
db = Chroma(
    embedding_function=embeddings,
    persist_directory=persistent_directory
)

# Only add documents if the directory is new (or if you want to reprocess)
if not os.path.exists(persistent_directory):
    print("Persistent directory does not exist. Initializing vector store...")

    loader = TextLoader(file_path)
    document = loader.load()

    raw_text = document[0].page_content
    docs = split_egyptian_civil_law(raw_text)

    batch_size = 50
    for i in range(0, len(docs), batch_size):
        batch = docs[i:i + batch_size]
        db.add_documents(batch)
        print(f"Processed {i + len(batch)} / {len(docs)} documents")

    db.persist()
    print("Vector store created and persisted.")

else:
    print("Persistent directory exists. Loaded existing vector store.")

C:\Users\seife\AppData\Local\Temp\ipykernel_18596\3453657790.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
C:\Users\seife\AppData\Local\Temp\ipykernel_18596\3453657790.py:15: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  db = Chroma(


Persistent directory exists. Loaded existing vector store.


In [4]:
def retrieve_articles(query: str, k: int = 5):
    """
    Retrieves the most relevant legal articles from the Egyptian Civil Law.
    Returns LangChain Documents with full hierarchy metadata.
    """
    docs = db.similarity_search(
        query,
        k=k,
        filter={"type": "article"}
    )
    return docs

In [5]:
def print_retrieved_articles(docs):
    for i, doc in enumerate(docs, 1):
        meta = doc.metadata
        print("=" * 80)
        print(f"[{i}] {meta.get('title')}")
        print(f"📘 Book: {meta.get('book')}")
        print(f"📂 Part: {meta.get('part')}")
        print(f"📄 Chapter: {meta.get('chapter')}")
        print("-" * 80)
        print(doc.page_content)  # preview only


In [12]:
query = "إذا تعارضت المادة 150 والمادة 147 في موضوع العقد، كيف يتم حل النزاع؟"
answer = retrieve_articles(query, k=5)
print_retrieved_articles(answer)


[1] المادة 147
📘 Book: الكتاب الأول – الالتزامات بوجه عام
📂 Part: الباب الأول – مصادر الالتزام
📄 Chapter: الفصل الأول – العقد
--------------------------------------------------------------------------------
المادة 147

147
العقد شريعة المتعاقدين، فلا يجوز نقضه أو تعديله إلا باتفاق الطرفين أو لأسباب يقررها القانون.

إذا طرأت حوادث استثنائية لم يكن في الوسع توقعها، وترتب على حدوثها أن تنفيذ الالتزام صار مرهقًا للمدين، جاز للقاضي بعد الموازنة بين مصالح الطرفين أن يرد الالتزام إلى الحد المعقول. ويكون أي اتفاق على خلاف ذلك باطلاً.
[2] المادة 149
📘 Book: الكتاب الأول – الالتزامات بوجه عام
📂 Part: الباب الأول – مصادر الالتزام
📄 Chapter: الفصل الأول – العقد
--------------------------------------------------------------------------------
المادة 149

149
– إذا تم العقد بطريق الإذعان واشتمل على شروط تعسفية، جاز للقاضي تعديلها أو إعفاء الطرف المذعن منها وفق العدالة، ويقع باطلاً كل اتفاق على خلاف ذلك.
[3] المادة 157
📘 Book: الكتاب الأول – الالتزامات بوجه عام
📂 Part: الباب الأول – مصادر الالتزام
📄 C

In [7]:
query = "ما شروط صحة الكفالة؟"
answer = retrieve_articles(query, k=5)
print_retrieved_articles(answer)


[1] المادة 776
📘 Book: الكتاب الثاني – العقود المسماة
📂 Part: الباب الخامس - الكفالة
📄 Chapter: الفصل الأول - أركان الكفالة
--------------------------------------------------------------------------------
المادة 776

776
لا تكون الكفالة صحيحة إلا إذا كان الإلزام المكفول صحيحًا.
[2] المادة 772
📘 Book: الكتاب الثاني – العقود المسماة
📂 Part: الباب الخامس - الكفالة
📄 Chapter: الفصل الأول - أركان الكفالة
--------------------------------------------------------------------------------
المادة 772

772
الكفالة عقد يكفل فيه شخص تنفيذ التزام إذا لم ينفذه المدين.
[3] المادة 773
📘 Book: الكتاب الثاني – العقود المسماة
📂 Part: الباب الخامس - الكفالة
📄 Chapter: الفصل الأول - أركان الكفالة
--------------------------------------------------------------------------------
المادة 773

773
لا تثبت الكفالة إلا بالكتابة، حتى لو كان الحائز يثبت الالتزام الأصلي بالبينة.
[4] المادة 780
📘 Book: الكتاب الثاني – العقود المسماة
📂 Part: الباب الخامس - الكفالة
📄 Chapter: الفصل الأول - أركان الكفالة
------------------

In [22]:
query = "ما المقصود بالالتزام؟"
answer = retrieve_articles(query, k=5)
print_retrieved_articles(answer)


[1] المادة 271
📘 Book: الكتاب الأول – الالتزامات بوجه عام
📂 Part: الباب الثاني - آثار الالتزام
📄 Chapter: الفصل الأول - الشرط والأجل
--------------------------------------------------------------------------------
المادة 271

271
يكون الالتزام لأجلٍ إذا كان انقضاؤه أو نفاذه مترتبًا على أمرٍ محقق الوقوع.

ويُعتبر الأمر محقق الوقوع متى كان وقوعه حتميًا، ولو لم يُعرف الوقت الذي يقع فيه.
[2] المادة 668
📘 Book: الكتاب الثاني – العقود المسماة
📂 Part: الباب الثاني - العقود الواردة على الانتفاع بالشيء
📄 Chapter: الفصل الأول - المقاولة والتزام المرافق العامة
--------------------------------------------------------------------------------
المادة 668

668
التزام المرافق العامة عقد يهدف إلى إدارة مرفق عام ذي صفة اقتصادية، بين جهة الإدارة المختصة وفرد أو شركة تتعهد باستغلال المرفق لفترة محددة.
[3] المادة 347
📘 Book: الكتاب الأول – الالتزامات بوجه عام
📂 Part: الباب الثاني - آثار الالتزام
📄 Chapter: الفصل الأول - الوفاء
--------------------------------------------------------------------------------


In [7]:
def audit_articles(docs, expected_last_article=1149):
    found_articles = {}
    duplicates = []

    for doc in docs:
        if doc.metadata.get("type") == "article":
            idx = doc.metadata.get("index")

            if idx is None:
                print("⚠️ Article with missing index:", doc.metadata)
                continue

            if idx in found_articles:
                duplicates.append(idx)
            else:
                found_articles[idx] = doc

    expected_set = set(range(1, expected_last_article + 1))
    found_set = set(found_articles.keys())

    missing = sorted(expected_set - found_set)
    extra = sorted(found_set - expected_set)

    print("=== ARTICLE AUDIT ===")
    print(f"Expected articles : {expected_last_article}")
    print(f"Found articles    : {len(found_set)}")
    print(f"Missing articles  : {len(missing)}")
    print(f"Duplicate articles: {len(duplicates)}")
    print(f"Extra articles    : {len(extra)}")

    return {
        "missing": missing,
        "duplicates": duplicates,
        "extra": extra
    }


In [8]:
report = audit_articles(docs)


=== ARTICLE AUDIT ===
Expected articles : 1149
Found articles    : 1093
Missing articles  : 56
Duplicate articles: 2
Extra articles    : 0


In [9]:
def print_ranges(numbers):
    if not numbers:
        print("No missing articles ✅")
        return

    ranges = []
    start = prev = numbers[0]

    for n in numbers[1:]:
        if n == prev + 1:
            prev = n
        else:
            ranges.append((start, prev))
            start = prev = n
    ranges.append((start, prev))

    print("Missing article ranges:")
    for r in ranges:
        if r[0] == r[1]:
            print(f"- المادة {r[0]}")
        else:
            print(f"- المواد {r[0]} → {r[1]}")


In [10]:
print_ranges(report["missing"])

Missing article ranges:
- المواد 54 → 80
- المواد 389 → 417


In [11]:
from collections import Counter

def audit_sections(docs):
    section_counter = Counter()

    for doc in docs:
        section_type = doc.metadata.get("type", "UNKNOWN")
        section_counter[section_type] += 1

    print("\n=== SECTION DISTRIBUTION ===")
    for section, count in section_counter.items():
        print(f"{section:10} : {count}")

    return section_counter


In [12]:
section_stats = audit_sections(docs)


=== SECTION DISTRIBUTION ===
preface    : 1
article    : 1095
part       : 15
chapter    : 52
book       : 4


In [13]:
def find_bad_articles(docs):
    bad = []

    for doc in docs:
        if doc.metadata.get("type") == "article":
            content = doc.page_content.strip()

            if not content.startswith("المادة"):
                bad.append(doc.metadata)

    print(f"\nMalformed article chunks: {len(bad)}")
    return bad

bad_articles = find_bad_articles(docs)


Malformed article chunks: 0


In [14]:
def expected_min_chunks(expected_articles):
    # At least:
    # 1 preface + articles + books + chapters + sections
    return expected_articles + 20  # conservative lower bound

print(
    "Expected MIN chunks:",
    expected_min_chunks(1149),
    "| Actual chunks:",
    len(docs)
)


Expected MIN chunks: 1169 | Actual chunks: 1167


In [15]:
from collections import defaultdict

def check_books(docs):
    book_titles = defaultdict(list)  # title -> list of indices in docs

    for i, doc in enumerate(docs):
        if doc.metadata.get("type") == "book":
            title = doc.metadata.get("title", "UNKNOWN")
            book_titles[title].append(i)

    duplicates = {title: idxs for title, idxs in book_titles.items() if len(idxs) > 1}

    print(f"Total book chunks found: {len(book_titles)}")
    print(f"Unique book titles   : {len(book_titles.keys())}")
    print(f"Duplicated books     : {len(duplicates)}")

    if duplicates:
        print("\n=== Duplicated books ===")
        for title, idxs in duplicates.items():
            print(f"Book '{title}' repeated {len(idxs)} times at chunk indices {idxs}")

    return book_titles, duplicates


In [16]:
book_titles, duplicates = check_books(docs)

Total book chunks found: 4
Unique book titles   : 4
Duplicated books     : 0


In [17]:
# Collect all book titles
book_titles = []

for doc in docs:
    if doc.metadata.get("type") == "book":
        title = doc.metadata.get("title", "UNKNOWN")
        book_titles.append(title)

# Remove duplicates while preserving order
seen = set()
unique_books = []
for t in book_titles:
    if t not in seen:
        unique_books.append(t)
        seen.add(t)

# Display all book titles
print("=== List of Book Titles ===")
for i, title in enumerate(unique_books, 1):
    print(f"{i}. {title}")

print(f"\nTotal unique books: {len(unique_books)}")


=== List of Book Titles ===
1. الكتاب الأول – الالتزامات بوجه عام
2. الكتاب الثاني – العقود المسماة
3. الكتاب الثالث - الحقوق العينية الأصلية
4. الكتاب الرابع - الحقوق العينية التبعية أو التأمينات العينية

Total unique books: 4


In [18]:
for doc in docs:
    print("Type:", doc.metadata["type"])
    print("Title:", doc.metadata["title"])
    print("Book:", doc.metadata.get("book"))
    print("Part:", doc.metadata.get("part"))
    print("Chapter:", doc.metadata.get("chapter"))
    print("Content:", doc.page_content[:60], "...\n")

Type: preface
Title: المقدمة
Book: None
Part: None
Chapter: None
Content: القانون رقم 131 لسنة 1948 بإصدار القانون المدني ...

Type: article
Title: المادة 1
Book: None
Part: None
Chapter: None
Content: المادة 1

1
يلغي القانون المدني المعمول به أمام المحاكم الوط ...

Type: article
Title: المادة 2
Book: None
Part: None
Chapter: None
Content: المادة 2

2
على وزير العدل تنفيذ هذا القانون ويعمل به ابتداء ...

Type: part
Title: الباب تمهيدي – أحكام عامة
Book: None
Part: الباب تمهيدي – أحكام عامة
Chapter: None
Content: الباب تمهيدي – أحكام عامة


 ...

Type: chapter
Title: الفصل الأول – القانون وتطبيقه
Book: None
Part: الباب تمهيدي – أحكام عامة
Chapter: الفصل الأول – القانون وتطبيقه
Content: الفصل الأول – القانون وتطبيقه

1- القانون والحق ...

Type: article
Title: المادة 1
Book: None
Part: الباب تمهيدي – أحكام عامة
Chapter: الفصل الأول – القانون وتطبيقه
Content: المادة 1

1
تسري النصوص التشريعية على جميع المسائل التي تتنا ...

Type: article
Title: المادة 2
Book: None
Part: الباب تمهيدي – أحك